# setup

In [ ]:
%matplotlib inline
import eelbrain as eel
import numpy as np
import scipy, pathlib, importlib, mne, time, os, sys
from tqdm import tqdm
import matplotlib.pyplot as plt
import preprocessing as pre
import models as md
import plotting
from pathnames import *
importlib.reload(pre)
importlib.reload(md)
importlib.reload(plotting)

mne.set_log_level(verbose='error')
verbose = False

# Load EEG data and preprocess

In [ ]:
ch = 'Cz'
refs = ['EXG1', 'EXG2'] # mastoids

preprocessed_path.mkdir(exist_ok=True, parents=True)
rewrite_output = False # set to true to overwrite subjects that have already been processed
subjects = [f'part_{i:03d}' for i in range(1, 26)]

for stimtype in ['in', 's']: # 'in': inserts, 's': sound-field
    speechfiles = [f'{stimtype}_speech_{i}a' for i in range(1,5)] + [f'{stimtype}_speech_{i}b' for i in range(1,5)]
    clickfile = f'{stimtype}_clicks_4c'

    print(speechfiles)

    # loop over subjects
    for subject in tqdm(subjects):
        filename = preprocessed_path / f'{subject}_{stimtype}_speech_{ch}_reref_ergs.pkl'
        if filename.exists() and not rewrite_output:
            print('skipping', subject, filename)
            continue

        subject_folder = rawdata_path / subject

        # speech files
        eegs = [] # eeg
        ergs = [] # Erg
        eegfs = [] # eeg file names
        fp_ergs = [] # first peak time in Erg channel (eeg and Erg start after this)
        for eegf in speechfiles:
            if subject == 'part_018' and eegf == 'in_speech_4b': # part_018 has missing data error in trial 4b
                continue
            if subject == 'part_024' and eegf in ['in_speech_2a', 'in_speech_4b']: # part_024 has flat ERG channels for trials 2a and 4b
                continue
            eegfile = subject_folder / f'P{subject[-3:]}C{eegf}.bdf'
            if not eegfile.exists():
                print(f'FILE {eegfile.stem} NOT FOUND')
                continue
            # load EEG, reref and extract channels
            eeg_a, erg_a, erg_start = pre.load_eeg(eegfile, ch=ch, refs=refs)
            eegs.append(eeg_a)
            eegfs.append(eegfile.stem)
            ergs.append(erg_a)
            fp_ergs.append(erg_start)
        eel.save.pickle(dict(eegs=eegs, filenames=eegfs, fp_ergs=fp_ergs, ergs=ergs), preprocessed_path / f'{subject}_{stimtype}_speech_{ch}_reref_ergs.pkl')

        # click file
        eegfile = rawdata_path / subject / f'P{subject[-3:]}C{stimtype}_clicks_4c.bdf'
        eeg_a, erg_a, erg_start = pre.load_eeg(eegfile, ch=ch, refs=refs) # preprocessing
        eel.save.pickle(dict(eeg=eeg_a, erg=erg_a, erg_start=erg_start), preprocessed_path / f'{subject}_{stimtype}_clicks_{ch}_reref_{"".join(refs)}.pkl')

        # also load io clicks
        if stimtype=='in': 
            eegfile = rawdata_path / subject / f'P{subject[-3:]}Cio_clicks_4c.bdf'
            eeg_a, erg_a, erg_start = pre.load_eeg(eegfile)
            eel.save.pickle(dict(eeg=eeg_a, erg=erg_a, erg_start=erg_start), preprocessed_path / f'{subject}_io_clicks_{ch}_reref_{"".join(refs)}.pkl')